# Planck Results Plotting

In [ ]:
import warnings
import functools
import pandas as pd
import matplotlib.ticker
import matplotlib.pyplot as plt
import nestcheck.diagnostics_tables
import nestcheck.estimators as e
import nestcheck.data_processing
import diagnostic.results_plots
%matplotlib inline

# Check matplotlib parameters
# ---------------------------
matplotlib_settings = {'text.usetex': True,
                       'font.family': ['serif'],
                       'font.serif': ['Times New Roman'],
                       'font.size': 8}
for key, value in matplotlib_settings.items():
    if matplotlib.rcParams.get(key) != value:
        print('{}={} - the paper plots use {}'.format(
            key, matplotlib.rcParams.get(key), value))

In [ ]:
planck_dict = {}
for nrepeats in [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35]:
    if nrepeats in [1, 2, 3, 4, 5, 6, 7, 28]:
        # reran due to accidentally using an old PolyChord version with bug
        inds = list(range(25, 50))
    else:
        inds = list(range(25))
    print('nrepeats={}'.format(nrepeats))
    file_roots = ['{}_repeats_{}'.format(nrepeats, i) for i in inds]
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message=r'for logl=')
        run_list = nestcheck.data_processing.batch_process_data(
            file_roots, base_dir='chains_planck', parallel=True, errors_to_handle=(ValueError, OSError))
    planck_dict[nrepeats] = run_list

In [ ]:
# Get the quantities we want to plot, and their latex names
with open('chains_planck/1_repeats_1.paramnames', 'r') as file:
    data = file.read()
latex_names = [string.split('\t')[1] for string in data.split('\n')[:-1]]
estimator_list = [e.logz]
estimator_names = [e.get_latex_name(e.logz)]
for ind in [0, 1, 3, 4, 5, 6, 7]:
    estimator_list.append(functools.partial(e.param_mean, param_ind=ind))
    estimator_names.append('$\overline{{{}}}$'.format(latex_names[ind]))

In [ ]:
n_simulate = 100
planck_results = {}
for key, run_list in planck_dict.items():
    save_name = 'cache/errors_df_planck_{}repeats_{}runs_{}sim'.format(
        key, len(inds), n_simulate)
    planck_results[key] = nestcheck.diagnostics_tables.run_list_error_summary(
        run_list, estimator_list, estimator_names, n_simulate=n_simulate,
        save_name=save_name, save=True, load=True)
planck_df_in = pd.concat(planck_results)
planck_df_in.index.names = ['nrepeats'] + list(planck_df_in.index.names[1:]) 
planck_df = planck_df_in
planck_df

In [ ]:
calculation_types = ['bootstrap std mean',
                     'implementation std',
                     'values std']
linestyles = ['-', '--', ':', '-.']  # to ensure linestyles match key, which is made as a seperate pdf
# Make the plot in two halves
assert len(planck_df.columns) == len(estimator_names) == 8
for i in range(2):
    df_temp = planck_df.loc[planck_df.index.get_level_values(0) >= 3]
    df_temp = df_temp[estimator_names[i * 4:(i + 1) * 4]]
    fig = diagnostic.results_plots.get_line_plot(
        df_temp, calculation_types, linestyles=linestyles,
        log_scale=False, left_margin=0.4)
    for ax in fig.axes:
        # standard form numbers on y labels
        ax.get_yaxis().set_major_formatter(matplotlib.ticker.FormatStrFormatter('%.e'))
    savename = 'plots/line_nrepeats_planck_25runs_{}sim_{}.pdf'.format(
        n_simulate, i + 1).replace(' ', '_')
    fig.savefig(savename)